[![Binder](https://mybinder.org/badge_logo.svg)](https://nbviewer.org/github/Sistemas-Multimedia/Sistemas-Multimedia.github.io/blob/master/milestones/gray_compression/deadzone.ipynb)

# Gray Image compression Using a Midrise Quantizer

In [ ]:
#!ln -sf ~/MRVC/src/debug.py .
!ln -sf ~/MRVC/src/logging_config.py .
!ln -sf ~/repos/quantization/quantizer.py .
!ln -sf ~/repos/quantization/midrise_quantizer.py .
!ln -sf ~/repos/quantization/distortion.py .
#!ln -sf ~/MRVC/src/image_3.py .
!ln -sf ~/MRVC/src/image_1.py .
!ln -sf ~/repos/quantization/information.py .

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.axes as ax
#plt.rcParams['text.usetex'] = True
#plt.rcParams['text.latex.preamble'] = [r'\usepackage{amsmath}'] #for \text command
import pylab
import math
import numpy as np
from scipy import signal
import cv2
import os
import midrise_quantizer as midrise
import distortion
#import image_3 as RGB_image
import image_1 as gray_image
import colored
import information

## Configuration

In [ ]:
# Prefix of the RGB image to be quantized.

home = os.environ["HOME"]
fn = home + "/MRVC/images/lena_bw/"

components = ['R', 'G', 'B']

quantizer = midrise

# Maybe this does not make sense in digital quantization (generate loops in the RD domain
# because the output number of quantization indexes is not a power of 2.)
#Q_steps = range(128, 0, -4)

Q_steps = [2**i for i in range(7, -1, -1)] # Quantization steps (simulating bit-plane encoding)

#quantizer = companded
#Q_steps = [2**i for i in range(16, -1, -1)] # Quantization steps (simulating bit-plane encoding)

print(Q_steps)

#RGB_image.write = RGB_image.debug_write # faster
#RGB_image.write = RGB_image.write # higher compression

gray_image.write = gray_image.debug_write # faster
#gray_image.write = gray_image.write # higher compression

Notice that non embbeded quatization (using steps thar are different from a power of 2) steps (can produce loops in the RD curves due to the non-linearity of the integer division performed in the quantization).

## Read the image and show it

In [ ]:
img = gray_image.read(fn, 0).astype(np.int16) # 8 bits/components is not sufficient to shift the components to [-128, 127]
print(img.dtype)
print(img.max(), img.min())
gray_image.show(img, fn + "000.png")

## Show some quantizations

In [ ]:
QS = 32 # Quantization Step

In [ ]:
Q = quantizer.Midrise_Quantizer(Q_step=QS, min_val=0, max_val=255)
print(Q.get_decision_levels())

In [ ]:
print(Q.quantize(np.array([127])))

In [ ]:
print(Q.quantize(np.array([128])))

In [ ]:
print(Q.quantize(np.array([256])))

In [ ]:
y, k = Q.quan_dequan(img)
print("Used quantization indexes:", np.unique(k))
gray_image.show_normalized(k, f"{quantizer.name} $\\Delta={QS}$")
gray_image.show(y, f"{quantizer.name} $\\Delta={QS}$")
print("MSE =", distortion.MSE(img, y))
print("SSIM =", distortion.SSIM(img, y))
print("entropy =", information.entropy(k.flatten()))

In [ ]:
Q2 = quantizer.Midrise_Quantizer2(Q_step=QS, min_val=0, max_val=255)

In [ ]:
print(Q2.quantize(np.array([127])))

In [ ]:
print(Q2.quantize(np.array([128])))

In [ ]:
print(Q2.quantize(np.array([129])))

In [ ]:
y2, k2 = Q2.quan_dequan(img)
print("Used quantization indexes:", np.unique(k2))
gray_image.show_normalized(k2, f"{quantizer.name} $\\Delta={QS}$")
gray_image.show(y, f"{quantizer.name} $\\Delta={QS}$")
print("MSE =", distortion.MSE(img, y))
print("SSIM =", distortion.SSIM(img, y2))
print("entropy =", information.entropy(k.flatten()))

## RD curve

In [ ]:
def RD_curve(img, Q_steps, quantizer):
    points = []
    for Q_step in Q_steps:
        Q = quantizer.Midrise_Quantizer(Q_step=Q_step, min_val=0, max_val=255)
        y, k = Q.quan_dequan(img)
        k = k.astype(np.uint8) # Only positive components can be written in an PNG file
        print("Quantization indexes: ", np.unique(k))
        #rate = common.bits_per_color_pixel(k, str(Q_step) + '_') # Remember that k has 3 components
        rate = gray_image.write(k, "/tmp/" + str(Q_step) + '_', 0)*8/k.size
        _distortion = distortion.RMSE(img, y)
        points.append((rate, _distortion))
        print(f"q_step={Q_step:>3}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
    return points

RD_points = RD_curve(img, Q_steps, quantizer)

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*RD_points), c='m', marker='x', label=f"{quantizer.name}", linestyle="dotted")
pylab.title(f"Rate/Distortion Performance ({quantizer.name})")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
pylab.legend(loc='upper right')
pylab.show()

In [ ]:
print(quantizer.name)
with open(f"{quantizer.name}_RD_points.txt", 'w') as f:
    for item in RD_points:
        f.write(f"{item[0]}\t{item[1]}\n")